# Exercises on 3D animations with matrix transformations

## Exercise 5.1
Write a function `infer_matrix(n, transformation)` that takes a dimension and a function that is a vector transformation assumed to be linear and returns an nxn square matrix that represents the linear transformation.

Note that the given `transformation` **has** to be linear, otherwise, the returned matrix will not represent the transformation.

We know that to completely specify a linear transformation, it is enough to specify how it transforms the vectors of the standard basis.

Thus, what we have to do is is apply the given transformation to the standard basis of the given dimension, collect the results in columns, and return that matrix.

Let's start with a supporting function that will return the standard basis vectors.

For the standard basis vectors, we need to come up with a collection of tuples on which the coordinate 1 is shifted in position for every vector.

```
n = 2:
    e1 = (1, 0)
    e2 = (0, 1)
    standard_basis = [e1, e2]

n = 3:
    e1 = (1, 0, 0)
    e2 = (0, 1, 0)
    e3 = (0, 0, 1)
    standard_basis = [e1, e2, e3]

n = 4:
    e1 = (1, 0, 0, 0)
    e2 = (0, 1, 0, 0)
    e3 = (0, 0, 1, 0)
    e4 = (0, 0, 0, 1)
    standard_basis = [e1, e2, e3, e4]
```

Now, we're beginning to see that we will need a couple of nested iterations: one over the number of dimensions given (number of standard basis vectors) and an inner one to build the elements of each of the vectors.

Let's start with a *procedural*, non-Pythonic way:

In [3]:
def standard_basis(dimension):
    standard_basis = []
    for i in range(0, dimension):
        e = []
        for j in range(0, dimension):
            if (i == j):
                e.append(1)
            else:
                e.append(0)
        standard_basis.append(e)
    return standard_basis

print(standard_basis(2))
print(standard_basis(3))
print(standard_basis(4))

[[1, 0], [0, 1]]
[[1, 0, 0], [0, 1, 0], [0, 0, 1]]
[[1, 0, 0, 0], [0, 1, 0, 0], [0, 0, 1, 0], [0, 0, 0, 1]]


That solution more or less works, but we are only using lists instead of tuples, and it cries for a more succinct implementation.

Let's give it a second try with the hint that list and tuple comprehensions in Python allows for using if conditions in their definition:

In [7]:
def standard_basis(dimension):
    standard_basis = [
        tuple(1 if i == j else 0 for j in range(0, dimension)) 
        for i in range(0, dimension)
        ]
    return standard_basis

print(standard_basis(2))
print(standard_basis(3))
print(standard_basis(4))

[(1, 0), (0, 1)]
[(1, 0, 0), (0, 1, 0), (0, 0, 1)]
[(1, 0, 0, 0), (0, 1, 0, 0), (0, 0, 1, 0), (0, 0, 0, 1)]


Note that for readability purposes, the author recommends defining a support function to return the standard basis vector, but either way is fine.



In [8]:
def standard_basis(dimension):
    def standard_basis_vector(i):
        return tuple(1 if i == j else 0 for j in range(0, dimension))
    standard_basis = [standard_basis_vector(i) for i in range(0, dimension)]
    return standard_basis

print(standard_basis(2))
print(standard_basis(3))
print(standard_basis(4))

[(1, 0), (0, 1)]
[(1, 0, 0), (0, 1, 0), (0, 0, 1)]
[(1, 0, 0, 0), (0, 1, 0, 0), (0, 0, 1, 0), (0, 0, 0, 1)]


In [ ]:
Now, we need to define our main function:

In [9]:
def infer_matrix(n, transformation):
    transformations_on_standard_basis = [transformation(ei) for ei in standard_basis(n)]
    return tuple(zip(*transformations_on_standard_basis))

Note that we had to use `zip(*matrix)` to convert the matrix rows into columns.

Let's validate it with a transformation like `rotate_z_by(pi / 2)`:

In [12]:
from my_transformations import rotate_z_by
from math import pi

infer_matrix(3, rotate_z_by(pi / 2))

((6.123233995736766e-17, -1.0, 0.0),
 (1.0, 1.2246467991473532e-16, 0.0),
 (0, 0, 1))

We can validate it also visually, by using this function to rotate our teapot:

In [19]:
from my_teapot import load_triangles
from my_draw_model import draw_model
from math import sin, cos
from my_transformations import rotate_z_by, polygon_map
from my_matrices import multiply_matrix_vector
from math import pi

def standard_basis(dimension):
    def standard_basis_vector(i):
        return tuple(1 if i == j else 0 for j in range(0, dimension))
    standard_basis = [standard_basis_vector(i) for i in range(0, dimension)]
    return standard_basis

def infer_matrix(n, transformation):
    transformations_on_standard_basis = [transformation(ei) for ei in standard_basis(n)]
    return tuple(zip(*transformations_on_standard_basis))

rotation_matrix = infer_matrix(3, rotate_z_by(pi / 2))

def matrix_rotate_z_by(angle_radians):
    matrix_transformation = infer_matrix(3, rotate_z_by(angle_radians))
    def new_function(v):
        return multiply_matrix_vector(matrix_transformation, v)
    return new_function

draw_model(polygon_map(matrix_rotate_z_by(pi / 2), load_triangles()))

SystemExit: 0

In [ ]:
Note that as this rotation matrix does not depend on time, the teapot is not animated, but rather rotated once.

# Exercise 5.2

What is the result of the following product?

$
\begin{pmatrix}
1.3 & -0.7 \\
6.5 & 3.2
\end{pmatrix}
\begin{pmatrix}
-2.5 \\
0.3
\end{pmatrix}
$

The result will be a column vector with dimension 2, with $ a_{11} = (1.3, -0.7) \cdot (-2.5, 0.3) $ and $ a_{21} = (6.5, 3.2) \cdot (-2.5, 0.3) $

$
\begin{pmatrix}
1.3 & -0.7 \\
6.5 & 3.2
\end{pmatrix}
\begin{pmatrix}
-2.5 \\
0.3
\end{pmatrix} = \begin{pmatrix}
-3.46 \\
-15.29
\end{pmatrix}
$


Let's validate it with our Python functions:

In [21]:
from my_matrices import multiply_matrix_vector

m = (
    (1.3, -0.7),
    (6.5, 3.2)
    )
v = (-2.5, 0.3)

print(multiply_matrix_vector(m, v))

(-3.46, -15.29)


# Exercise 5.3

Write a `random_matrix(...)` function that generates matrices of a specified size with random whole number entries. Use the function to generate five pairs or 3x3 matrices. Multiply each of the pairs together by hand for practice and then check your work with the `matrix_multiply(...)` function.

Let's define the `random_matrix(...)` function first:

From the initial chapter we know that we can get random ints using:

```python
import random

print(random.randint(0, 10))
```

This can also be written as:

```python
from random import randint

print(randint(0, 10))
```

In [3]:
from random import randint

def random_matrix(num_rows, num_cols, min_int_inc=0, max_int_inc=10):
    def get_random_in_range(min_val, max_val):
        def new_function():
            return randint(min_val, max_val + 1)
        return new_function
    rand = get_random_in_range(min_int_inc, max_int_inc)
    result = tuple(
        tuple(rand() for i in range(0, num_cols))
        for j in range(0, num_rows)
    )
    return result

print(random_matrix(2, 2))
print(random_matrix(4, 3))
print(random_matrix(2, 4, -5, 5))


((2, 10), (0, 4))
((2, 10, 11), (8, 6, 11), (11, 3, 11), (4, 3, 8))
((-1, 5, 3, 0), (-4, 6, -1, -1))


Now, let's generate 5 pairs of 3-by-3 matrices:

In [7]:
from my_matrices import random_matrix

for i in range(0, 5):
    print(random_matrix(3, 3, min_int_inc=-2, max_int_inc=2))
    print(random_matrix(3, 3, min_int_inc=-2, max_int_inc=2))
    print()


((2, -1, -2), (0, -2, 1), (0, 0, -1))
((0, 0, 0), (1, 2, 0), (2, 3, 2))

((2, 3, 2), (1, -1, -2), (3, 0, 3))
((1, 1, -2), (2, -1, -2), (0, 0, -1))

((0, 3, 0), (-2, 3, -1), (-1, -2, 3))
((1, 1, 0), (3, -1, -1), (-1, 2, -2))

((0, 0, 0), (-1, 3, 3), (2, 3, 0))
((-1, 2, 1), (2, -2, -2), (-2, -1, 0))

((-1, 1, 3), (-1, 0, 2), (2, 1, 0))
((0, 2, 1), (-1, 3, 2), (3, -2, 0))



In [9]:
from my_matrices import matrix_multiply

matrix_multiply(((0, 1, 0), (-2, -1, 0), (3, 2, 1)), ((3, 3, 0), (0, -2, -2), (1, 0, -2)))

((0, -2, -2), (-6, -4, 2), (10, 5, -6))

# Exercise 5.4

For each of the matrices from the previous exercise, multiply them in the opposite order. Do you get the same result?

In [10]:
from my_matrices import matrix_multiply

matrix_multiply(((3, 3, 0), (0, -2, -2), (1, 0, -2)), ((0, 1, 0), (-2, -1, 0), (3, 2, 1)))

((-6, 0, 0), (-2, -2, -2), (-6, -3, -2))

So, no, the result is not the same.

# Exercises 5.5

In either 2D or 3D, there is a boring but important transformation called the *identity transformation* that takes in a vector and returns the same vector as output. This transformation is linear because it preserves (obviously) a linear combination of vectors. What are the matrices representing the identity transformation in 2D and 3D respectively?

We know from the concepts section that in order to compute the matrix representing a transformation we apply the transformation to the vectors of the standard basis and place the result in the columns of the matrix.

Therefore, let $ I_{d=2} $ the identity transformation for the 2D plane, $ I_{d=3} $ the identity transformation for the 3D space.

Therefore:
$
I_{d=2} = \begin{pmatrix}
1 & 0 \\
0 & 1
\end{pmatrix} \\
I_{d=3} = \begin{pmatrix}
1 & 0 & 0 \\
0 & 1 & 0 \\
0 & 0 & 1
\end{pmatrix} \\
$

# Exercise 5.6

Apply the matrix ((2, 1, 1), (1, 2, 1), (1, 1, 2)) to all the vectors defining the teapot. What happens to the teapot and why?

In [16]:
from my_teapot import load_triangles
from my_draw_model import draw_model
from my_transformations import polygon_map
from my_matrices import multiply_matrix_vector

transformation_matrix = ((2, 1, 1), (1, 2, 1), (1, 1, 2))

def apply_matrix_transformation(matrix):
    def new_function(v):
        return multiply_matrix_vector(matrix, v)
    return new_function

draw_model(polygon_map(apply_matrix_transformation(transformation_matrix), load_triangles()))

SystemExit: 0

The teapot seems to have been stretched out into the region where x, y and z are all positive. This happens because the transformation is changing:

$
A(e1) = (2, 1, 1) \\
A(e2) = (1, 2, 1) \\
A(e3) = (1, 1, 2)
$

That is, all the vectors will be stretched into that region where x > 0, y > 0, z > 0

# Exercise 5.7

Implement multiply_matrix_vector in a different way using two nested comprehensions: one traversing the rows of the matrix, and one traversing the entries of each rows



It is asked to perform the sums and products as if we were using the mnemonic recipes. The only benefit is that it is a zero-dependency solution

In [22]:
def multiply_matrix_vector(matrix,vector):
    return tuple(
        sum(vector_entry * matrix_entry for vector_entry, matrix_entry in zip(row,vector)       ) 
        for row in matrix)

# (0, -6, 10)
print(multiply_matrix_vector(
    ((0, 1, 0),
    (-2, -1, 0),
    (3, 2, 1)
    ),
    (3, 0, 1))
)

(0, -6, 10)


# Exercise 5.8

Implement `multiply_matrix_vector(...)` yet another way using the fact that the output coordinates are the dot products of the input matrix rows with the input vector.

In [25]:
from my_vectors import dot

def multiply_matrix_vector(m, v):
    return tuple(dot(row, v) for row in m)

# (0, -6, 10)
print(multiply_matrix_vector(
    ((0, 1, 0),
    (-2, -1, 0),
    (3, 2, 1)
    ),
    (3, 0, 1))
)

(0, -6, 10)


# Exercise 5.9

We know that any linear transformation can be represented by a matrix. Demonstrate the converse fact:
> all matrices represent linear transformations. That is, show that matrix multiplication preserves sums and scalar multiples.

Hint: start with the explicit formulas for multiplying a 2D vector by a 2x2 matrix and multiplying a 3D vector by a 3x3 matrix.

The algebraic demonstration is easy but long, the idea is to demonstrate that:

Let $ A $ a matrix, $ v $, $ w $ vectors such that:
$
A = \begin{pmatrix}
a_{00} & a_{01} & a_{02} \\
a_{10} & a_{11} & a_{12} \\
a_{20} & a_{21} & a_{22}
\end{pmatrix} \\
v = (v_x, v_y, v_z), \hspace{1ex} w = (w_x, w_y, w_z)
$

You have to demonstrate that:
$
A(a \cdot v + b \cdot w) = a \cdot Av + b \cdot Bw
$

which is evident as:
$
A(a \cdot v + b \cdot w) = \begin{pmatrix}
a_{00} & a_{01} & a_{02} \\
a_{10} & a_{11} & a_{12} \\
a_{20} & a_{21} & a_{22}
\end{pmatrix}
\begin{pmatrix}
a \cdot v_x + b \cdot w_x \\
a \cdot v_y + b \cdot w_y \\
a \cdot v_z + b \cdot w_z
\end{pmatrix}
$

As a consequence, all 3x3 matrices are linear transformations, as they preserve linear combinations.


## Exercise 5.10

Consider that following matrices representing transformations:

$
A = \begin{pmatrix}
1 & 1 & 0 \\
1 & 0 & 1 \\
1 & -1 & 1
\end{pmatrix}, \hspace{1ex} B = \begin{pmatrix}
0 & 2 & 1 \\
0 & 1 & 0 \\
1 & 0 & -1
\end{pmatrix}
$

Write a function `compose_a_b(...)` that executes the composition of the linear transformation for $ A $ and the linear transformation for $ B $.
Then use the `infer_matrix(...)` function on the result to check that those results match.

We know from the concept section that composition of linear transformations is the same as matrix multiplication.

Therefore:

In [1]:
from my_matrices import *

def compose_a_b(a, b):
    composition_matrix = matrix_multiply(a, b)
    def new_function(v):
        return multiply_matrix_vector(composition_matrix, v)
    return new_function

a = (
    (1, 1, 0),
    (1, 0, 1),
    (1, -1, 1)
)

b = (
    (0, 2, 1),
    (0, 1, 0),
    (1, 0, -1)
)

# ((0, 3, 1), (1, 2, 0), (1, 1, 0))
print(matrix_multiply(a, b))

# (0, 1, 1)
print(compose_a_b(a, b)((1, 0, 0)))

# (3, 2, 1)
print(compose_a_b(a, b)((0, 1, 0)))

# (1, 0, 0)
print(compose_a_b(a, b)((0, 0, 1)))

# ((0, 3, 1), (1, 2, 0), (1, 1, 0))
print(infer_matrix(3, compose_a_b(a, b)))


((0, 3, 1), (1, 2, 0), (1, 1, 0))
(0, 1, 1)
(3, 2, 1)
(1, 0, 0)
((0, 3, 1), (1, 2, 0), (1, 1, 0))


## Exercise 5.12

Find two 2x2 matrices, neither of which is the identity matrux, but whose product is the identity matrix.

We need to find two matrices $ A $ and $ B $ such that:

$
AB = \begin{pmatrix}
1 & 0 \\
0 & 1 
\end{pmatrix}
$

And we know that:

$
AB = \begin{pmatrix}
a_{00} & a_{01} \\
a_{10} & a_{11}
\end{pmatrix}
\begin{pmatrix}
b_{00} & b_{01} \\
b_{10} & b_{11}
\end{pmatrix} = \begin{pmatrix}
1 & 0 \\
0 & 1 
\end{pmatrix}
$

That will however lead to quite a messy set of equations, so there is a much better way to think of solving this problem using the properties of linear transformations
> multiplying two matrices together is the same as composing linear transformations &mdash; if two composed transformations lead to the identity matrix, it means they return the original vector.

Thus, we can think of matrices representing transformations that when composed, return the original vector:
+ $ A $ scales the vector by 2, $ B $ scales the vector by $ \frac{1}{2} $
+ $ A $ rotates the vector 90° and $ B $ rotates the vector 270°

For the first case:
$
A = \begin{pmatrix}
2 & 0 \\
0 & 2
\end{pmatrix}, \hspace{1ex} B = \begin{pmatrix}
\frac{1}{2} & 0 \\
0 & \frac{1}{2}
\end{pmatrix}
$

For the second case:
$
A = \begin{pmatrix}
0 & -1 \\
1 & 0
\end{pmatrix}, \hspace{1ex} B = \begin{pmatrix}
0 & 1 \\
-1 & 0
\end{pmatrix}
$

| NOTE: |
| :---- |
| Remember that to come up with the matrix representing a linear transformation, you have consider that the first column of the matrix represent the effect of the transformation on the $ e1 $ vector, the second column is the result of the transformation on $ e2 $, etc. |

## Exercise 5.12

We can multiply a square matrix by itself any number of times. We can think of successive matrix multiplications as *"raising a matrix to a power"*. For example, $ AA = A² $, $ AAA = A³ $, etc.

Write a `matrix_power(power, matrix)` function that raises a matrix to the specified whole number power.

In [3]:
from my_matrices import *

def matrix_power(power, matrix):
    temp_result = matrix
    for i in range(0, power):
        temp_result = matrix_multiply(matrix, matrix)
    return temp_result

a = (
    (1, 2),
    (3, 4)
)

# ((7, 10), (15, 22))
print(matrix_power(2, a))

# ((7, 10), (15, 22))
print(matrix_multiply(a, a))

((7, 10), (15, 22))
((7, 10), (15, 22))


In [ ]:
It can also be coded using recursion:

In [6]:
from my_matrices import *

def matrix_power(power, matrix):
    if power == 1:
        return matrix
    else:
        return matrix_multiply(matrix_power(power - 1, matrix), matrix)
a = (
    (1, 2),
    (3, 4)
)

# ((7, 10), (15, 22))
print(matrix_power(2, a))

# ((7, 10), (15, 22))
print(matrix_multiply(a, a))

((7, 10), (15, 22))
((7, 10), (15, 22))
